In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from darts_pro.base import DartBoard
from darts_pro.dqn import (
    Agent,
    PureGreedyStrategy,
    LinearNetwork
)
from darts_pro.games.o_one import X01GameState, X01TrainingLoop, X01TrainingConfig

In [12]:
N_EPISODES = 200
NETWORK_SHAPE = [128, 128]

In [4]:
board = DartBoard.get_default_dartboard(False)
input_size = X01GameState({0: 0}, 301, 0, 0, '', 0).to_tensor().shape[0]
policy_network = LinearNetwork(input_size, len(board.indexed_targets), NETWORK_SHAPE)
target_network = LinearNetwork(input_size, len(board.indexed_targets), NETWORK_SHAPE)
target_network.load_state_dict(policy_network.state_dict())
target_network.eval()

config = X01TrainingConfig()

In [8]:
loop = X01TrainingLoop(N_EPISODES, policy_network, target_network, config, board=board)

In [9]:
loop.run()

Finished game 0. Final score 0.0. Total Reward -1.0
Finished game 1. Final score 0.0. Total Reward -1.0
Finished game 2. Final score 0.0. Total Reward 0.5133420832795048
Finished game 3. Final score 0.0. Total Reward -1.0
Finished game 4. Final score 0.0. Total Reward 0.26352009446574204
Finished game 5. Final score 0.0. Total Reward 0.3073568677250236
Finished game 6. Final score 0.0. Total Reward -1.0
Finished game 7. Final score 0.0. Total Reward 0.6634204312890623
Finished game 8. Final score 0.0. Total Reward -1.0
Finished game 9. Final score 0.0. Total Reward 0.3405616262881148
Finished game 10. Final score 0.0. Total Reward -1.0
Finished game 11. Final score 0.0. Total Reward 0.5987369392383787
Finished game 12. Final score 0.0. Total Reward 0.6302494097246091
Finished game 13. Final score 0.0. Total Reward -1.0
Finished game 14. Final score 0.0. Total Reward -1.0
Finished game 15. Final score 0.0. Total Reward 0.37735360253530725
Finished game 16. Final score 0.0. Total Reward 

KeyboardInterrupt: 

## Testing After Training

In [47]:
board = DartBoard.get_default_dartboard(False)
all_actions = list(board.indexed_targets.keys())

new_agent = Agent(PureGreedyStrategy(), all_actions)
dummy_state = X01GameState({0: 5}, 301, 17, 0, '', 0).to_tensor()[None]
act = new_agent.select_action(dummy_state, policy_network)
print(board.indexed_targets[act])

Target(value=2, multiplier=3, is_bullseye=False)
